In [7]:
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
import ipywidgets as widgets
from ipywidgets import BoundedFloatText, BoundedIntText, Button, Checkbox, DatePicker, HBox, Label, Layout, Output, RadioButtons, VBox
from IPython.display import display

# Construct a birthdatetime from inputs
def build_birthdatetime():
    return datetime(bd.value.year, bd.value.month, bd.value.day, bt_h.value, bt_m.value, bt_s.value)

# Convert datetime to hours
def datetime_to_hours(datetime1):
    hours = datetime1.days * 24
    hours += datetime1.hours
    hours += datetime1.minutes / 60
    hours += datetime1.seconds / 3600
    return np.round(hours, 3)

# Get datetime until now
def get_hours_to_date(birthdatetime):
    now = datetime.now()
    diff = relativedelta(now, birthdatetime)
    return datetime_to_hours(diff)

# Get coeff for a column with a given DataFrame
def get_lin_coeff(df, col):
    coeffs = [(0, 0)]
    pts = zip(df.index, df[col])
    pt1 = next(pts)
    for pt in pts:
        m, b = linear_eq(pt1, pt)
        coeffs.append((m, b))
#         print(pt1[0], pt[0])
#         plt.plot(np.arange(pt1[0], pt[0]+1), m*np.arange(pt1[0], pt[0]+1) + b)
#         print(f'{m:.3}x + {b:.3}')
        pt1 = pt
    coeffs[0] = coeffs[1]
    return coeffs

# Get coeff for any 2 given points
def linear_eq(pt1, pt2):
    x1, y1 = pt1
    x2, y2 = pt2
    m = (y2 - y1) / (x2 - x1)
    b = y1 - m * x1
    return np.round(m, 4), np.round(b, 4)

def select_photo_curve():
    has_risk = False
    for cb in cb_risks:
        has_risk = cb.value or has_risk
        
    if ga.value == '35-37 6/7 weeks':
        if has_risk:
            coeff_curve = 'Higher Coeff'
        else:
            coeff_curve = 'Medium Coeff'
    else:
        if has_risk:
            coeff_curve = 'Medium Coeff'
        else:
            coeff_curve = 'Lower Coeff'
    return coeff_curve

# Get the zone of the given SB and PA for Jaundice
def get_zone(sb, pa):
    interval = np.where(jaundice_df.index <= pa)[0][-1]
#     print(interval)
    
    hi_coeff = jaundice_df['High Intermediate Coeff'].iloc[interval]
    li_coeff = jaundice_df['Low Intermediate Coeff'].iloc[interval]
    l_coeff = jaundice_df['Low Coeff'].iloc[interval]

    upper_hi = hi_coeff[0] * pa + hi_coeff[1]
    upper_li = li_coeff[0] * pa + li_coeff[1]
    upper_l = l_coeff[0] * pa + l_coeff[1]
    
    ok_hi = sb < upper_hi
    ok_li = sb < upper_li
    ok_l = sb < upper_l
    if ok_l and ok_li and ok_hi:
        msg = 'Low Risk Zone'
    elif ok_li and ok_hi:
        msg = 'Low Intermediate Risk Zone'
    elif ok_hi:
        msg = 'High Intermediate Risk Zone'
    else:
        msg = 'High Risk Zone'
    return msg

def need_photo(sb, pa, col, margin=0.5):
    interval = np.where(photo_df.index <= pa)[0][-1]
#     print(interval)
    coeff = photo_df[col].iloc[interval]
#     print(coeff)
    upper = coeff[0] * pa + coeff[1]
    if sb >= upper:
        msg = 'Required Phototherapy'
    elif upper - sb <= margin:
        msg = f'Optional Phototherapy\n({upper - sb:.3} mg/dl lower than the guideline)'
    else:
        msg = 'No Phototherapy'
    return msg

# Read xlsx and get linear coeff
# mostly for visualization and interaction
if __name__ == "__main__":
    stat = Output()
    
    # Plot the Jaundice graph with given inputs
    @stat.capture()
    def gitGud(b):
        def plotPhototherapy(sb, pa):
            curve = select_photo_curve()
            zone = need_photo(sb, pa, col=curve)
            print('Phototherapy: ' + zone)
            show_all = cb_sp.value

            plt.figure(figsize=(15,8))
        #     ax = plt.subplot(121)
            plt.scatter(pa, sb, s=80, color='red', label=f'Serum Bilirubin: {sb} mg/dl\nPostnatal Age: {pa} hours\nZone: {zone}')
            if cb_sp.value or curve == 'Lower Coeff':
                plt.plot(photo_df.index, photo_df['Lower Risk'], ':', color='green', label='Infants at Lower Risk (>= 38 weeks and well)')
            if cb_sp.value or curve == 'Medium Coeff':
                plt.plot(photo_df.index, photo_df['Medium Risk'], '--', color='orange', label='Infants at Medium Risk (>= 38 weeks + risk factors OR 35-37 6/7 weeks and well)')
            if cb_sp.value or curve == 'Higher Coeff':
                plt.plot(photo_df.index, photo_df['Higher Risk'], color='red', label='Infants at Higher Risk (35-37 6/7 weeks + risk factors)')
            plt.ylabel('Total Serum Bilirubin (mg/dl)',fontsize=15)
            plt.xlabel('Postnatal Age (hours)',fontsize=15)
            plt.title('Phototherapy Graph',fontsize=15)
            plt.xticks(photo_df.index)
        #     ax.set_ylabel('Total Serum Bilirubin (mg/dl)',fontsize=15)
        #     ax.set_xlabel('Postnatal Age (hours)',fontsize=15)
        #     ax.set_title('Phototherapy Graph',fontsize=20)
        #     ax.set_xticks(photo_df.index)
            plt.legend(fontsize=10)
            plt.grid()
            with photo_out:
                plt.show()
                
            ################ plotPhototherapy ENDS HERE ####################

        def plotJaundice(sb, pa):
            zone = get_zone(sb, pa)
            print('\nJaundice: ' + zone)

            plt.figure(figsize=(15,8))
        #     ax = plt.subplot(122)
            plt.scatter(pa, sb, s=80, color='red', label=f'Serum Bilirubin: {sb} mg/dl\nPostnatal Age: {pa} hours\nZone: {zone}')
            plt.plot(jaundice_df.index, jaundice_df['Low Risk'], '--', color='green', label='Low')
            plt.plot(jaundice_df.index, jaundice_df['Low Intermediate Risk'], '--', color='orange', label='Low Intermediate')
            plt.plot(jaundice_df.index, jaundice_df['High Intermediate Risk'], '--', color='red', label='High Intermediate')
            plt.ylabel('Serum Bilirubin (mg/dl)',fontsize=15)
            plt.xlabel('Postnatal Age (hours)',fontsize=15)
            plt.title('Neonatal Jaundice Graph',fontsize=15)
            plt.xticks(jaundice_df.index)
        #     ax.set_ylabel('Serum Bilirubin (mg/dl)',fontsize=15)
        #     ax.set_xlabel('Postnatal Age (hours)',fontsize=15)
        #     ax.set_title('Neonatal Jaundice Graph',fontsize=20)
        #     ax.set_xticks(jaundice_df.index)
            plt.legend(fontsize=10)
            plt.grid()
            with jaundice_out:
                plt.show()
        ################ plotJaundice ENDS HERE ####################
        jaundice_out.clear_output()
        photo_out.clear_output()
        stat.clear_output()
    #     out.clear_output()
        serum_b = sb.value
        if not (0 <= serum_b):
            print('ERROR: Serum Bilirubin (mg/dl) must be a non-negative real number.')
            return
        postn = get_hours_to_date(build_birthdatetime())
        if not (12 <= postn):
            print('ERROR: Postnatal Age (hours) must be greater than or equal to 12.')
            return
    #     plt.figure(figsize=(16,10))
        plotPhototherapy(serum_b, postn)
        plotJaundice(serum_b, postn)
    #     plt.tight_layout()
    #     with out:
    #         plt.show()

    ################## MAIN STARTS HERE ##########################
    
    photo_df = pd.read_excel('./Phototherapy.xlsx', index_col='Postnatal Age') 
    photo_df['Lower Coeff'] = get_lin_coeff(photo_df, 'Lower Risk')
    photo_df['Medium Coeff'] = get_lin_coeff(photo_df, 'Medium Risk')
    photo_df['Higher Coeff'] = get_lin_coeff(photo_df, 'Higher Risk')
    jaundice_df = pd.read_excel('./Jaundice Risk.xlsx', index_col='Postnatal Age')
    jaundice_df['Low Coeff'] = get_lin_coeff(jaundice_df, 'Low Risk')
    jaundice_df['Low Intermediate Coeff'] = get_lin_coeff(jaundice_df, 'Low Intermediate Risk')
    jaundice_df['High Intermediate Coeff'] = get_lin_coeff(jaundice_df, 'High Intermediate Risk')

    twelve_hr_ago = datetime.now() - timedelta(hours=12)
    
#     align_args = dict(
#         _css = (('.widget-label', 'min-width', '20ex'),),
#         margin = '0px 0px 5px 12px'
#     )
    
    sb = BoundedFloatText(value=0.1, min=0, layout=Layout(width='auto'))
#     pa = BoundedFloatText(value=12, layout=Layout(width='20%'))
    bd = DatePicker(value=twelve_hr_ago.date(), layout=Layout(width='auto'))
    
    bt_h = BoundedIntText(value=twelve_hr_ago.hour, min=0, max=24, step=1, layout=Layout(width='auto'))
    bt_m = BoundedIntText(value=twelve_hr_ago.minute, min=0, max=60, step=1, layout=Layout(width='auto'))
    bt_s = BoundedIntText(value=twelve_hr_ago.second, min=0, max=60, step=1, layout=Layout(width='auto'))
    
    ga = RadioButtons(options=['>= 38 weeks', '35-37 6/7 weeks'])
    risk_factors = ['Isoimmune hemolytic disease (ABO)',
             'G6PD deficiency',
             'Sepsis',
             'Acidosis',
             'Asphyxia',
             'Temperature instability',
             'Significant lethargy',
             'Albumin < 3 mg/dl (if measured)'
            ]
    cb_risks = []
    for factor in risk_factors:
        cb = Checkbox(description=factor, layout=Layout(width='auto'))
        cb_risks.append(cb)
    
    btn = Button(description='Do me a favor!')
    btn.on_click(gitGud)
    
    box_sb = HBox([Label(value='Serum Bilirubin (mg/dl): '), sb])
#     box_pa = HBox([Label(value='Postnatal Age (hours): '), pa])
    box_bd = HBox([Label(value='Birthdate: '), bd])
    box_bt = HBox([Label(value='Birth time: '), 
                   Label(value='24-Hour: '),
                   bt_h,
                   Label(value='Minute: '),
                   bt_m,
                   Label(value='Second: '),
                   bt_s])
    box_ga = HBox([Label(value='Gestational age: '), ga])
    box_risks = VBox([Label(value='Risk factors:', layout=Layout(width='auto'))] + cb_risks)
    
    cb_sp = Checkbox(value=True, description='Show all phototherapy curves', layout=Layout(width='auto'))
    box_cb_sp = VBox([Label(value='Graphing option:', layout=Layout(width='auto')), cb_sp])
    box_left = VBox([box_sb, box_bd, box_bt, box_ga])
#     box_right = box_risks
    box = VBox([box_left, box_risks, box_cb_sp])
    
#     out = Output()
    photo_out = Output()
    jaundice_out = Output()
    
    display(box)
    display(btn)
    display(stat)
#     display(out)
    display(photo_out)
    display(jaundice_out)

Button(description='Do me a favor!', style=ButtonStyle())

Output()

Output()

Output()